In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_log_error
import xgboost
# from catboost import CatBoostRegressor, Pool
import h2o
from sklearn import metrics
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, VotingRegressor, AdaBoostRegressor, GradientBoostingRegressor, ExtraTreesRegressor, StackingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.svm import SVR

In [5]:
!pip install h2o

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
h2o.init(
    nthreads=-1,     # number of threads when launching a new H2O server
    max_mem_size=12  # in gigabytes
)

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.18" 2023-01-17; OpenJDK Runtime Environment (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1); OpenJDK 64-Bit Server VM (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.9/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpgfbpgb4t
  JVM stdout: /tmp/tmpgfbpgb4t/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpgfbpgb4t/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,05 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.2
H2O_cluster_version_age:,16 days
H2O_cluster_name:,H2O_from_python_unknownUser_m925fj
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,12 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
train = h2o.import_file('drive/MyDrive/House-price-prediction-in-Moscow-/train.csv')

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [9]:
train_as_df = h2o.as_list(train,use_pandas=True)
train_as_df = train_as_df.fillna('None')

train_as_df.head(2)

,Id,Room,Price,Size,Balcony,Floor,FloorsTotal,Walls,Age,Lift,District,Okrug
0,1,5,210020000,198,балкон,1,2,None,вторичка,None,Сокол,САО
1,2,2,17630000,64,None,3,8,монолитный,вторичка,грузовой,Зюзино,ЮЗАО


In [10]:
train_as_df.set_index('Id', inplace=True)

train_as_df['log_Price'] = np.log1p(train_as_df['Price']) #log to make values smaller
target = 'log_Price'
train_as_df.drop(['Price'], axis=1, inplace=True)

In [11]:
train = h2o.H2OFrame(train_as_df)
x = train.columns
y = target
x.remove(y)

# test = h2o.H2OFrame(validation)
# x_val = validation.columns
# y_val = target

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [12]:
from h2o.automl import H2OAutoML


aml = H2OAutoML(max_models=11, seed=54)
aml.train(x=x, y=y, training_frame=train)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


key,value
Stacking strategy,cross_validation
Number of base models (used / total),9/11
# GBM base models (used / total),5/5
# XGBoost base models (used / total),3/3
# DRF base models (used / total),1/2
# GLM base models (used / total),0/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5
Metalearner fold_column,None


In [13]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

model_id,rmse,mse,mae,rmsle,mean_residual_deviance
StackedEnsemble_AllModels_1_AutoML_1_20230326_91518,0.178073,0.0317101,0.121624,0.0098743,0.0317101
StackedEnsemble_BestOfFamily_1_AutoML_1_20230326_91518,0.178257,0.0317755,0.12156,0.00988767,0.0317755
GBM_3_AutoML_1_20230326_91518,0.181278,0.0328616,0.124502,0.010059,0.0328616
GBM_4_AutoML_1_20230326_91518,0.182654,0.0333624,0.124077,0.0101264,0.0333624
GBM_2_AutoML_1_20230326_91518,0.183919,0.0338263,0.126423,0.010199,0.0338263
DRF_1_AutoML_1_20230326_91518,0.186219,0.0346774,0.12464,0.0103115,0.0346774
GBM_5_AutoML_1_20230326_91518,0.186257,0.0346918,0.128875,0.0103338,0.0346918
GBM_1_AutoML_1_20230326_91518,0.186903,0.0349328,0.128147,0.0103377,0.0349328
XGBoost_2_AutoML_1_20230326_91518,0.190432,0.0362644,0.131211,0.0105614,0.0362644
XGBoost_3_AutoML_1_20230326_91518,0.191856,0.0368088,0.133166,0.0106362,0.0368088


In [15]:
test = h2o.import_file('drive/MyDrive/House-price-prediction-in-Moscow-/test.csv')
test_as_df = h2o.as_list(test,use_pandas=True)
test_as_df = test_as_df.fillna('None')

test_as_df.set_index('Id', inplace=True)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [16]:
y_pred = aml.leader.predict(test)

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


In [17]:

# y_pred = aml.predict(test)
# y_pred3 = model3.predict(X_val)
# y_pred4 = model4.predict(X_val)
y_pred[target] = y_pred['predict']

y_pred = h2o.as_list(y_pred,use_pandas=True)

# print('RMSE:', np.sqrt(metrics.mean_squared_error(y_val, y_pred[target])))

In [18]:
y_pred.drop('predict', axis=1, inplace=True)

In [19]:
# y_pred = np.expm1(y_pred)
# # # Save the predictions
# submission_df = pd.DataFrame({'Id': test_df['Id'], 'Price': y_pred2})
# submission_df.to_csv('submission2.csv', index=False)
submission_df = pd.DataFrame({'Id': test_as_df.index, 'Price': np.expm1(y_pred)[target]})
# submission_df.to_csv('submission_cboost.csv', index=False)

In [20]:
submission_df.to_csv('best_submission.csv', index=False)